In [1]:
import numpy as np
from random import random, seed
from copy import deepcopy

In [20]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_matrix(cls, matrix, vertices = None):
        if (vertices is None) or (len(vertices) != len(matrix)):
            vertices = [*range(1, len(matrix) + 1)]
        return cls.from_dict(cls._matrix_to_dict(matrix, vertices))

    # two private methods matrix <-> dictionaries
    def _matrix_to_dict(matrix, vertices: list) -> dict:
        """
        Converts a graph given as an adjacency matrix to a graph in dict form.
        """
        res_dict = {}
        for i, v in enumerate(vertices):
            neighbours = [vertices[j] for j, edge in enumerate(matrix[i]) if edge]
            res_dict[v] = neighbours
        return res_dict

    def _dict_to_matrix(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to its adjacency matrix.
        """
        n = len(_dict)
        vertices = [*_dict.keys()]
        matrix = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (vertices.index(u), vertices.index(v))
            for u, row in _dict.items() for v in row
        ]:
            matrix[u][v] += 1
        return matrix

    def vertices(self) -> list:
        """
        Returns list of vertices of the graph.
        """
        return [*self.graph.keys()]

    def matrix(self) -> np.array:
        """
        Returns the adjacency matrix of the graph.
        """
        return self._dict_to_matrix(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    # The following is for free thanks to the above
    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")  # open textfile for writing
        file.write(str(self))
        file.close()

    # Modyfying graphs
    def add_vertex(self, vertex):
        """
        Adds a new vertex to the graph.
        """
        if vertex not in self.graph:
            self.graph[vertex] = []

    def del_vertex(self, vertex):
        """
        Removes a vertex from the graph.
        """
        if vertex in self.graph:
            self.graph.pop(vertex)
            for u in self.graph:
                if vertex in self.graph[u]:
                    self.graph[u].remove(vertex)

    def add_arc(self, arc):
        """
        Given pair of vertices (arc variable) add an arc to the graph
        We consider simple, directed graphs.
        """
        u, v = arc
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Given pair of vertices (edge variable) add an edge to existing graph.
        We consider simple, undirected graphs, as symmetric digraphs without loops.
        """
        u, v = edge
        if u == v:
            raise ValueError("Loops are not allowed!")
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    # reading from a file
    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Read the graph from file, that in each line contains either
        the description of a vertex (one word) or
        the description of an edge/arc (at least 2 words).
        The resulting graph is returned as a neighbourhood list.
        Variable "filename" contains the whole path to the file.
        """
        graph = Graph()
        file = open(filename, "r")          # open the file to read
        for line in file:                   # for each line of the file
          words = line.strip().split()      # splits the line into words
          if len(words) == 1:               # one word - vertex description
            graph.add_vertex(words[0])
          elif len(words) >= 2:             # at least two words, first two are the edge description
            if directed:
              graph.add_arc([words[0], words[1]])
            else:
              graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(n: int, p: float):
        """
        Creates a random graph in G(n, p) model.
        """
        rand_graph = Graph()
        for i in range(1, n + 1):
            rand_graph.add_vertex(i)
            for j in range(1, i):
                if random() < p:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(n: int):
        """
        Creates a cycle C_n on n vertices
        """
        cycle = Graph()
        for i in range(n-1):
          cycle.add_edge([i+1, i+2])
        cycle.add_edge([1, n])
        return cycle


    def Prufer(self):
      """
      Returns the Prufer code of a tree.
      It is necessary that the graph is a tree (it is not checked).
      Result is given as a string (empty for trees on 1 or 2 vertices).
      """
      tr = deepcopy(self.graph)   # copy of a tree, as we destroy it
      code = ""
      for i in range(len(self.graph) - 2):
        for x in sorted(tr):
          if len(tr[x]) == 1:   # least leaf
            break
        v = tr[x][0]            # the unique neighbour of x
        code = code + f"{v} "
        tr[v].remove(x)         # remove x from neighbours of v
        tr.pop(x)               # remove x from the tree
      return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Creating a tree from a Prufer code.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]   # code as a list of numbers
        n = len(clist) + 2                  # number of vertices
        vert = [*range(1, n+1)]             # list of numbers 1..n
        for v in vert:
          tree.add_vertex(v)
        for i in range(n-2):
          for x in vert:
            if not x in clist:    # x - least leaf
              break
          v = clist.pop(0)    # remove the first element from the code - the neighbour of x
          tree.add_edge((x, v))
          vert.remove(x)
        tree.add_edge(vert)
        return tree


    def connected_components(self):
      """
      Looks for connected components of undirected graph.
      Returns a list of its vertex-sets.
      Remark: the first element contains the set of all graph vertices
      """
      def DFS(u):
        """
        Deep first search (as internal method).
        """
        for w in self.graph[u]:
          if w not in VT[0]:      # w - not visited yet
            VT[0].add(w)          # already visited
            VT[-1].add(w)         # w - in the last connected component
            DFS(w)

      """
      VT - list of vertex sets VT[i] for i > 0 - is a vertex set of i-th connected component
      VT[0] - is a vertex set of the spanning forest (or during the algorithm list of visited vertices).
      """
      VT = [set([])]
      for v in self.graph:
        if v not in VT[0]:      # v is not visited
          VT[0].add(v)
          VT.append(set([v]))   # statring point of new conected component
          DFS(v)
      return VT

    def preorder(self, v, visited=None):
        # first we print a given vertex, then we traverse the subtree rooted in it
        """
        Prints the vertices of the graph in preorder traversal starting from vertex v.
        """
        if visited is None:
            visited = set()

        # Visit the current node
        visited.add(v)
        print(v, end=' ')

        # Recursively visit all unvisited neighbors
        for neighbor in self.graph[v]:
            if neighbor not in visited:
                self.preorder(neighbor, visited)


    def postorder(self, v, visited=None):
        # postorder — first we traverse the subtree rooted a given vertex, then we list the vertex
        """
        Prints the vertices of the graph in postorder traversal starting from vertex v.
        """
        if visited is None:
            visited = set()

        # Mark the current node as visited
        visited.add(v)

        # Recursively visit all unvisited neighbors first
        for neighbor in self.graph[v]:
            if neighbor not in visited:
                self.postorder(neighbor, visited)

        # Print the current node after visiting its neighbors
        print(v, end=' ')


    def connected_components_graphs(self):
        """
        Returns a list of Graph objects, each representing a connected component of the original graph.
        """
        components = self.connected_components()  # Get the list of connected components as vertex sets
        component_graphs = []

        for component in components[1:]:  # Skip the first element as it contains all vertices
            subgraph = Graph()            # Create a new Graph instance
            for vertex in component:
                subgraph.graph[vertex] = self.graph[vertex]
#           equivalent
#                subgraph.add_vertex(vertex)
#                for neighbor in self.graph[vertex]:
#                    if neighbor in component:
#                        subgraph.add_edge([vertex, neighbor])
            component_graphs.append(subgraph)

        return component_graphs


    @staticmethod
    def random_bipartite_graph(m, n, p):
        """
        Generates a random bipartite graph with m + n vertices.
        The two sets U and V have m and n vertices respectively.
        Each edge between a vertex in U and a vertex in V is included with probability p.
        """
        bipartite_graph = Graph()
        U = [f"U{i}" for i in range(1, m + 1)]  # Label vertices in set U as U1, U2, ..., Um
        V = [f"V{i}" for i in range(1, n + 1)]  # Label vertices in set V as V1, V2, ..., Vn

        # Add vertices to the graph
        for vertex in U + V:
            bipartite_graph.add_vertex(vertex)

        # Add edges between vertices in U and V with probability p
        for u in U:
            for v in V:
                if random.random() < p:
                    bipartite_graph.add_edge([u, v])

        return bipartite_graph


    def distance(self, v):
      """
      Computes distances from vertex v to each vertex reachable from v.
      It uses a BFS approach.
      Result is given as a dictionary of distances
      """
      dist = {v:0}    # starting point of a dictionary
      queue = [v]
      while len(queue) > 0:
        u = queue.pop(0)
        for w in self.graph[u]:
          if not w in dist:
            dist[w] = dist[u] + 1
            queue.append(w)
      return dist
    
    def all_pairs_shortest_paths_bfs(self):
        """
        Computes shortest paths between all pairs of vertices using BFS.
        Returns a dictionary where dist[u][v] gives the shortest distance from u to v.
        """
        def bfs(start):
            dist = {start: 0}
            queue = [start]
            while queue:
                u = queue.pop(0)
                for v in self.graph[u]:
                    if v not in dist:
                        dist[v] = dist[u] + 1
                        queue.append(v)
            return dist
    
        vertices = self.vertices()
        dist = {u: {v: float('inf') for v in vertices} for u in vertices}
        for u in vertices:
            dist[u].update(bfs(u))
    
        return dist

    def floyd_warshall(self):
        """
        Floyd-Warshall algorithm to find shortest paths between all pairs of vertices.
        Returns a dictionary where dist[u][v] gives the shortest distance from u to v.
        """
        vertices = self.vertices()
        n = len(vertices)
        dist = {u: {v: float('inf') for v in vertices} for u in vertices}

        # Initialize distances using adjacency information
        for u in vertices:
            dist[u][u] = 0
            for v in self.graph[u]:
                dist[u][v] = 1  # Edge weight is 1 (small-world networks are typically unweighted)

        # Floyd-Warshall algorithm
        for k in vertices:
            for i in vertices:
                for j in vertices:
                    if dist[i][j] > dist[i][k] + dist[k][j]:
                        dist[i][j] = dist[i][k] + dist[k][j]

        return dist

    @staticmethod
    # Create a small-world network
    def create_small_world_graph(n, k, p):
        """
        Generates a small-world graph using the Watts-Strogatz model.
        - n: Number of vertices
        - k: Each vertex is connected to k nearest neighbors in a ring topology
        - p: Probability of rewiring each edge
        """
        graph = Graph()
        for i in range(n):
            graph.add_vertex(i)
            for j in range(1, k // 2 + 1):
                neighbor = (i + j) % n
                graph.add_edge([i, neighbor])
                if random.random() < p:
                    new_neighbor = random.choice([x for x in range(n) if x != i])
                    graph.add_edge([i, new_neighbor])
        return graph

1. With a large number of vertices, depth-first search encounters the problem of recursion depth. Prepare the ConnectedComponentsBFS function, which will use breadth-first search to determine strongly connected components. Alternatively, you can try to write DFS without using recursive calls.

In [ ]:
# DFS non-recursive
def ConnectedComponentsGraphs(self):
    con_com = []
    visited = set()
    
    def dfs(v, component):
        stack = [v]
        while stack:
            u = stack.pop()
            if u not in visited:
                visited.add(u)
                component[u] = [neighbor for neighbor in self.graph[u] if neighbor not in visited]
                stack.extend(component[u])

    # back to the main attraction
    for vertex in self.graph:
        if vertex not in visited:
            component = {}                  # creating a new component as an adj list
            dfs(vertex, component)          # find all vertices in this component
            con_com.append(Graph.from_dict(component))  #converting to Graph object and append
    
    return con_com


2. The example presented during classes (simulation of a small-world phenomenon) can be performed more efficiently using the Floyd-Warshal algorithm (for shortest paths). Check how much faster this method of implementation is.

In [ ]:
def all_pairs_shortest_paths_bfs(self):
        def bfs(start):
            dist = {start: 0}
            queue = [start]
            while queue:
                u = queue.pop(0)
                for v in self.graph[u]:
                    if v not in dist:
                        dist[v] = dist[u] + 1
                        queue.append(v)
            return dist
    
        vertices = self.vertices()
        dist = {u: {v: float('inf') for v in vertices} for u in vertices}
        for u in vertices:
            dist[u].update(bfs(u))
    
        return dist

def floyd_warshall(self):
    """
    Floyd-Warshall algorithm to find shortest paths between all pairs of vertices.
    Returns a dictionary where dist[u][v] gives the shortest distance from u to v.
    """
    vertices = self.vertices()
    n = len(vertices)
    dist = {u: {v: float('inf') for v in vertices} for u in vertices}
    # initialize distances using adjacency information
    for u in vertices:
        dist[u][u] = 0
        for v in self.graph[u]:
            dist[u][v] = 1
    # the algorithm
    for k in vertices:
        for i in vertices:
            for j in vertices:
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    return dist

@staticmethod
def create_small_world_graph(n, k, p):
    """
    Generates a small-world graph using the Watts-Strogatz model.
    - n: Number of vertices
    - k: Each vertex is connected to k nearest neighbors in a ring topology
    - p: Probability of rewiring each edge
    """
    graph = Graph()
    for i in range(n):
        graph.add_vertex(i)
        for j in range(1, k // 2 + 1):
            neighbor = (i + j) % n
            graph.add_edge([i, neighbor])
            if random.random() < p:
                new_neighbor = random.choice([x for x in range(n) if x != i])
                graph.add_edge([i, new_neighbor])
    return graph

In [ ]:
small_world_graph = Graph.create_small_world_graph(n=100, k=10, p=0.1)

In [ ]:
import time

small_world_graph = Graph.create_small_world_graph(n=100, k=10, p=0.1)

# runtime for BFS-based shortest paths
start_time = time.time()
bfs_distances = small_world_graph.all_pairs_shortest_paths_bfs()
bfs_time = time.time() - start_time
#print(f"BFS-based approach runtime: {bfs_time:.4f} seconds")

# runtime for Floyd-Warshall algorithm
start_time = time.time()
fw_distances = small_world_graph.floyd_warshall()
fw_time = time.time() - start_time
#print(f"Floyd-Warshall approach runtime: {fw_time:.4f} seconds")

print("Comparison")
print(f"BFS Time: {bfs_time:.4f} s")
print(f"Floyd-Warshall Time: {fw_time:.4f} s")


Comparison
BFS Time: 0.0178 s
Floyd-Warshall Time: 0.2725 s


3. Write the TopologicalSort function, which performs topological sorting of a directed graph (a slightly more difficult task, but the pseudocode was given in the lecture, and all the necessary elements — shown during classes).

In [ ]:
def topological_sort(self):
    visited = {v: False for v in self.vertices()}  # track visited vertices
    processed = {v: False for v in self.vertices()}  # track processed vertices
    topo_order = []  # store the topologically sorted order

    def dfs(vertex):
        visited[vertex] = True  # check the vertex as visited
        for neighbor in self.graph.get(vertex, []):     # recursively visit all the unvisited neighbors
            if not visited[neighbor]:
                dfs(neighbor)
                
        processed[vertex] = True  # check the vertex as processed
        topo_order.append(vertex)  # add the vertex to the topological order
        
    for vertex in self.vertices():      # start DFS for each unvisited vertex
        if not visited[vertex]:
            dfs(vertex)
    return topo_order[::-1]  # reverse the order before returning to get the correct topological sort
